In [1]:
""" Task: An HR department wants to use Alteryx to quickly understand the reporting structure for employees across the organization. 

The Input source contains 5 employees and an identifier that uniquely identifies the individual and the manager they report to. 

The goal is to create a hierarchy field identifying each relationship between employee and manager(s). 
For example, a Director reports directly to the Vice President which is 1 level up. 
The Director is then 2 levels away from the CEO (in this data set). As a result the hierarchy identifier represents how many levels removed the employee is from management team they report into."""

In [ ]:
import pandas as pd

# Create Dictionary - The Human Resources Data
HR_data = {
    'employee': ['Analyst', 'Manager', 'Director', 'Vice President', 'CEO'],
    'id': [3, 2, 1, 4, 5],
    'man_id': [2, 1, 4, 5,None]
}

# Convert DataType - HR_data - Dictionary: -> DataFrame
df = pd.DataFrame(HR_data)

# Adjust column - df "man_id" - Float -> Integer --- Comment = Since man_id has Null-Values, we replace them with -1, because -1 is not appearing in the id column
df['man_id'] = df['man_id'].fillna(-1)
df['man_id']=df['man_id'].astype(int)


In [19]:
# Create DataFrame - result - Purpose:An empty DataFrame for storing the end result
result =pd.DataFrame()

# Loop through every Row
for i,r in df.iterrows():
        # Create Value - hierarchy - Purpose:Counts how many levels are between the employee and the manager
        hierarchy = 0

        # Convert DataType - r - Series -> DataFrame --- Comment: We also restructure it, so it will be joinable
        r= r.to_frame().T
        
        """Idea: Join the man_id of the row with the id of the original DataFrame. 
        Then we get the Role of the immediately manager and the man_id. 
        Then we join the new man_id again with the original DataFrame and get the 2-Level-Up-Manager. 
        We repeat it through the while Loop until it can not find a manager again."""

        #Loop as long as necessary
        while True:
            # Create DataFrame - joined_table - Purpose: Will be used for further joins.
            joined_table = pd.merge(r,df,left_on='man_id',right_on='id',how='inner')
            # End Loop - if there are no rows anymore after the join. (Because the CEO does not have a Manager)
            if len(joined_table)==0:
                  break
            # Create DataFrame - relationship_table - Purpose: Contains the employee role and the name of the Manager role. Will be used in the output.
            relationship_table = joined_table.iloc[:,[0,3]]

            # Filter columns - joined_Table --- Comment: contains only employee, id and man_id
            joined_table = joined_table.iloc[:,[0,1,5]]

            # Create Column - relationship_table "Hierarchy"
            hierarchy=hierarchy+1
            relationship_table["Hierarchy"]=[hierarchy]

            # Adjust DataFrame - relationship_table - Rename Headers 
            relationship_table=relationship_table.rename(columns={'employee_x':'Employee','employee_y':'Manager'})

            # Adjust DataFrame - joined_Table - Purpose: The joined_Table will be used again as Input in the whilloop. Therefore it has to have the same Headers.
            joined_table.columns = joined_table.columns.str.replace('_y', '').str.replace('_x', '')

            # Adjust Table - result - Purpose: Collecting the results from the relationship_table after iteration
            result = pd.concat([result,relationship_table],ignore_index=True)

            # Adjust Table - r - Purpose: Adjusting the Input for the while-loop
            r=joined_table

print(result)

         Employee         Manager  Hierarchy
0         Analyst         Manager          1
1         Analyst        Director          2
2         Analyst  Vice President          3
3         Analyst             CEO          4
4         Manager        Director          1
5         Manager  Vice President          2
6         Manager             CEO          3
7        Director  Vice President          1
8        Director             CEO          2
9  Vice President             CEO          1
